# Índices y Tipología de la Complejidad Electoral 2020: Secciones Electorales
### INFOTEC-Maestría en Ciencia de Datos e Información
#### Autor: Miguel David Alvarez Hernández (mdalvarezh@gmail.com)

<br>

## <center>Procesamiento de datos y construcción de indicadores<center>

In [1]:
import datetime
now = datetime.datetime.now()
print ("Última versión:")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

Última versión:
2020-06-25 15:31:57


## Setup

In [2]:
import pandas as pd  
import numpy as np
import pandas_profiling
from itertools import combinations 
from dateutil.parser import parse 
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import plotly.express as px

## Importación de datos

In [3]:
#datos con el área de las secciones (estimacion feb-2020 con la cartografía de las manzanas)
data_area = pd.read_csv("Datos_secc_area_19.csv", dtype={'Area_Km2':float}) 

#series de tiempo de LNE y PE por seccion (enero 2019 - diciembre 2019)
data_lne = pd.read_csv("Datos_secc2019.csv", parse_dates=['DATE'], dtype={'PE':float, 'LNE':float})

#datos con el tipo de sección (clasificación de la Direccion de Cartografia)
data_tipo_secc = pd.read_csv('Datos_tipo_seccion.txt', sep="|")

#nomenclatura de los estados
data_nom_edo = pd.read_csv('Datos_Nomenclatura_EDO.csv', encoding='utf-8')

#datos con los nombres de los municipios
data_nom_mun = pd.read_csv('Datos_Nomenclatura_MUN.csv', encoding='utf-8')

In [4]:
print(data_tipo_secc.dtypes)
data_tipo_secc

EDO              int64
DTO              int64
MUN              int64
SECC             int64
TIPO_SECCION    object
dtype: object


,EDO,DTO,MUN,SECC,TIPO_SECCION
0,1,1,2,338,M
1,1,1,2,339,M
2,1,1,2,340,R
3,1,1,2,341,M
4,1,1,2,342,R
...,...,...,...,...,...
68431,32,4,57,530,M
68432,32,4,57,531,U
68433,32,4,57,532,U
68434,32,4,57,533,U


In [5]:
print(data_area.dtypes)
data_area

EDO           int64
DTO           int64
MUN           int64
SECC          int64
Area_Km2    float64
dtype: object


,EDO,DTO,MUN,SECC,Area_Km2
0,1,1,2,338,0.506492
1,1,1,2,339,0.692053
2,1,1,2,340,0.990536
3,1,1,2,341,0.507441
4,1,1,2,342,1.437017
...,...,...,...,...,...
61035,32,4,57,529,0.545010
61036,32,4,57,530,0.757383
61037,32,4,57,531,0.548031
61038,32,4,57,532,0.149622


In [6]:
print(data_lne.dtypes)
data_lne

DATE    datetime64[ns]
EDO              int64
DTO              int64
MUN              int64
SECC             int64
PE             float64
LNE            float64
dtype: object


,DATE,EDO,DTO,MUN,SECC,PE,LNE
0,2019-01-01,1,3,1,1,2163.0,2128.0
1,2019-01-01,1,3,1,2,903.0,891.0
2,2019-01-01,1,3,1,3,1710.0,1689.0
3,2019-01-01,1,3,1,4,1413.0,1397.0
4,2019-01-01,1,3,1,5,788.0,782.0
...,...,...,...,...,...,...,...
821125,2019-12-01,32,4,57,530,1671.0,1649.0
821126,2019-12-01,32,4,57,531,1601.0,1581.0
821127,2019-12-01,32,4,57,532,1081.0,1065.0
821128,2019-12-01,32,4,57,533,1183.0,1164.0


In [7]:
print(data_nom_edo.dtypes)
print(data_nom_mun.dtypes)

EDO               int64
NOMBRE_ESTADO    object
dtype: object
EDO                  int64
MUN                  int64
NOMBRE_MUNICIPIO    object
dtype: object


## Limpieza de datos

In [8]:
#ver si hay valores nulos en las series de tiempo data_lne
data_lne.isnull().sum()

DATE    0
EDO     0
DTO     0
MUN     0
SECC    0
PE      0
LNE     0
dtype: int64

In [9]:
#ver si hay valores nulos
data_area.isnull().sum()

EDO         0
DTO         0
MUN         0
SECC        0
Area_Km2    0
dtype: int64

In [10]:
#ver si hay valores nulos
data_tipo_secc.isnull().sum()

EDO             0
DTO             0
MUN             0
SECC            0
TIPO_SECCION    0
dtype: int64

In [11]:
#se eliminan las filas con valores vacíos
#data_lne.dropna()

In [12]:
#Se agrupan los datos por secciones
datos_lne = data_lne.groupby(['EDO','DTO','MUN','SECC']) #se agrupa con las columnas EDO, DTO, MUN, SECC
datos_lne.head() #se imprimen los primeros diez valores 

,DATE,EDO,DTO,MUN,SECC,PE,LNE
0,2019-01-01,1,3,1,1,2163.0,2128.0
1,2019-01-01,1,3,1,2,903.0,891.0
2,2019-01-01,1,3,1,3,1710.0,1689.0
3,2019-01-01,1,3,1,4,1413.0,1397.0
4,2019-01-01,1,3,1,5,788.0,782.0
...,...,...,...,...,...,...,...
342136,2019-05-01,32,4,17,1905,1232.0,1218.0
342137,2019-05-01,32,4,17,1906,704.0,694.0
342138,2019-05-01,32,4,17,1907,717.0,714.0
342139,2019-05-01,32,4,17,1908,1069.0,1057.0


In [13]:
#datos de enero de todas las secciones
enero=datos_lne.first() #first para tomar los primeros valores correspondientes a enero
enero2019=enero.drop(['DATE'], axis=1) #eliminamos DATE
enero_2019 = enero2019.reset_index() #recuperamos el índice
enero_2019

,EDO,DTO,MUN,SECC,PE,LNE
0,1,1,2,338,1803.0,1790.0
1,1,1,2,339,1713.0,1693.0
2,1,1,2,340,1741.0,1717.0
3,1,1,2,341,2059.0,2036.0
4,1,1,2,342,1553.0,1513.0
...,...,...,...,...,...,...
68423,32,4,57,530,1608.0,1586.0
68424,32,4,57,531,1542.0,1525.0
68425,32,4,57,532,1060.0,1042.0
68426,32,4,57,533,1123.0,1106.0


In [14]:
#datos de diciembre de todas las secciones
diciembre=datos_lne.last()
diciembre2019=diciembre.drop(['DATE'], axis=1)
diciembre_2019 = diciembre2019.reset_index() #recuperamos el índice
diciembre_2019

,EDO,DTO,MUN,SECC,PE,LNE
0,1,1,2,338,1786.0,1770.0
1,1,1,2,339,1724.0,1707.0
2,1,1,2,340,1771.0,1751.0
3,1,1,2,341,2074.0,2041.0
4,1,1,2,342,1559.0,1520.0
...,...,...,...,...,...,...
68423,32,4,57,530,1671.0,1649.0
68424,32,4,57,531,1601.0,1581.0
68425,32,4,57,532,1081.0,1065.0
68426,32,4,57,533,1183.0,1164.0



## Elaboración de los 7 indicadores base (_feature engineering_)

In [15]:
#calculamos los coeficientes de variacion del PE y la LNE para todas las secciones
datos_vc = datos_lne.agg({'PE': lambda x: np.std(x)/np.mean(x),'LNE': lambda x: np.std(x)/np.mean(x)}).rename(columns={'PE': 'Coef_Var_PE', 'LNE': 'Coef_Var_LNE'})
datos_vc

Coef_Var_PE  Coef_Var_LNE
EDO DTO MUN SECC                           
1   1   2   338      0.009274      0.014711
            339      0.006067      0.010667
            340      0.007424      0.013348
            341      0.006535      0.012118
            342      0.013673      0.020742
...                       ...           ...
32  4   57  530      0.013450      0.014736
            531      0.012040      0.012231
            532      0.006251      0.008482
            533      0.018088      0.018581
            548      0.013194      0.016949

[68428 rows x 2 columns]

In [16]:
#calculamos los coeficientes de autocorrelacion (lag=1month) para todas las secciones
datos_autcorr = datos_lne.agg({'PE': lambda x: x.autocorr(lag=1),'LNE': lambda x: x.autocorr(lag=1)}).rename(columns={'PE': 'Autocorr_PE', 'LNE': 'Autocorr_LNE'})
datos_autcorr

Autocorr_PE  Autocorr_LNE
EDO DTO MUN SECC                           
1   1   2   338      0.689191      0.581325
            339      0.717164      0.158740
            340      0.827905      0.152959
            341      0.635793      0.435180
            342      0.634410      0.367652
...                       ...           ...
32  4   57  530      0.955188      0.952950
            531      0.974564      0.961922
            532      0.934510      0.681005
            533      0.985049      0.928739
            548      0.598121      0.835766

[68428 rows x 2 columns]

In [17]:
#calculamos la razón media de LNE/PE para todas las secciones
data_lne['Razon_LNE_PE'] = data_lne['LNE']/data_lne['PE'] #calculamos la razón para todas las secciones y meses
datos_ratio=data_lne.groupby(['EDO','DTO','MUN','SECC'])['Razon_LNE_PE'].mean() 
datos_ratio

EDO  DTO  MUN  SECC
1    1    2    338     0.991580
               339     0.990970
               340     0.989805
               341     0.987758
               342     0.984600
                         ...   
32   4    57   530     0.988422
               531     0.989747
               532     0.986724
               533     0.986229
               548     0.990182
Name: Razon_LNE_PE, Length: 68428, dtype: float64

In [18]:
#calculamos la tasa de crecimiento de la LNE entre enero2019 y diciembre2019
diciembre_2019['TC_LNE_2019'] = (diciembre_2019['LNE'] - enero_2019['LNE'])/enero_2019['LNE']
diciembre_2019

,EDO,DTO,MUN,SECC,PE,LNE,TC_LNE_2019
0,1,1,2,338,1786.0,1770.0,-0.011173
1,1,1,2,339,1724.0,1707.0,0.008269
2,1,1,2,340,1771.0,1751.0,0.019802
3,1,1,2,341,2074.0,2041.0,0.002456
4,1,1,2,342,1559.0,1520.0,0.004627
...,...,...,...,...,...,...,...
68423,32,4,57,530,1671.0,1649.0,0.039723
68424,32,4,57,531,1601.0,1581.0,0.036721
68425,32,4,57,532,1081.0,1065.0,0.022073
68426,32,4,57,533,1183.0,1164.0,0.052441


## Función para calcular el indicador _Proportional Variability_

In [19]:
#Función para calcular PV
def PVar(lista):
    n = len(lista)
    CC= 2/(n*(n-1))
    sumaD = sum([(abs(i-j)/max(i,j)) for i,j in list(combinations(lista, 2))])
    return CC*sumaD

In [20]:
#ejemplo
array = np.arange(20).reshape(4,5)
print(array)
lista= array[:,1]
lista

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


array([ 1,  6, 11, 16])

In [21]:
PVar(lista)

0.6786616161616161

In [22]:
#calculamos la proportional variability para cada sección
datos_PV = datos_lne.agg({'PE': lambda x: PVar(x),'LNE': lambda x: PVar(x)}).rename(columns={'PE': 'Var_Prop_PE', 'LNE': 'Var_Prop_LNE'})
datos_PV

Var_Prop_PE  Var_Prop_LNE
EDO DTO MUN SECC                           
1   1   2   338      0.011123      0.017569
            339      0.007416      0.010046
            340      0.008319      0.013217
            341      0.007678      0.014344
            342      0.016305      0.024695
...                       ...           ...
32  4   57  530      0.015733      0.017848
            531      0.014507      0.014973
            532      0.007321      0.010120
            533      0.021717      0.022744
            548      0.014626      0.020291

[68428 rows x 2 columns]

## Unión de los datasets

In [23]:
#se unen coeficiente de variación y proportional variability
datos1 = datos_vc.join(datos_PV, lsuffix='_caller', rsuffix='_other')
datos1

Coef_Var_PE  Coef_Var_LNE  Var_Prop_PE  Var_Prop_LNE
EDO DTO MUN SECC                                                      
1   1   2   338      0.009274      0.014711     0.011123      0.017569
            339      0.006067      0.010667     0.007416      0.010046
            340      0.007424      0.013348     0.008319      0.013217
            341      0.006535      0.012118     0.007678      0.014344
            342      0.013673      0.020742     0.016305      0.024695
...                       ...           ...          ...           ...
32  4   57  530      0.013450      0.014736     0.015733      0.017848
            531      0.012040      0.012231     0.014507      0.014973
            532      0.006251      0.008482     0.007321      0.010120
            533      0.018088      0.018581     0.021717      0.022744
            548      0.013194      0.016949     0.014626      0.020291

[68428 rows x 4 columns]

In [24]:
#se agrega autocorrelacion
datos2019_secc = datos1.join(datos_autcorr, lsuffix='_caller', rsuffix='_other')
datos2019_secc

Coef_Var_PE  Coef_Var_LNE  Var_Prop_PE  Var_Prop_LNE  \
EDO DTO MUN SECC                                                         
1   1   2   338      0.009274      0.014711     0.011123      0.017569   
            339      0.006067      0.010667     0.007416      0.010046   
            340      0.007424      0.013348     0.008319      0.013217   
            341      0.006535      0.012118     0.007678      0.014344   
            342      0.013673      0.020742     0.016305      0.024695   
...                       ...           ...          ...           ...   
32  4   57  530      0.013450      0.014736     0.015733      0.017848   
            531      0.012040      0.012231     0.014507      0.014973   
            532      0.006251      0.008482     0.007321      0.010120   
            533      0.018088      0.018581     0.021717      0.022744   
            548      0.013194      0.016949     0.014626      0.020291   

                  Autocorr_PE  Autocorr_LNE  
EDO DTO MUN SECC                             
1   1   2   338      0.689191      0.581325  
            339      0.717164      0.158740  
            340      0.827905      0.152959  
            341      0.635793      0.435180  
            342      0.634410      0.367652  
...                       ...           ...  
32  4   57  530      0.955188      0.952950  
            531      0.974564      0.961922  
            532      0.934510      0.681005  
            533      0.985049      0.928739  
            548      0.598121      0.835766  

[68428 rows x 6 columns]

In [25]:
#se agrega razon media LNE/PE
datos2019_secc = datos2019_secc.join(datos_ratio, lsuffix='_caller', rsuffix='_other')
datos2019_secc

Coef_Var_PE  Coef_Var_LNE  Var_Prop_PE  Var_Prop_LNE  \
EDO DTO MUN SECC                                                         
1   1   2   338      0.009274      0.014711     0.011123      0.017569   
            339      0.006067      0.010667     0.007416      0.010046   
            340      0.007424      0.013348     0.008319      0.013217   
            341      0.006535      0.012118     0.007678      0.014344   
            342      0.013673      0.020742     0.016305      0.024695   
...                       ...           ...          ...           ...   
32  4   57  530      0.013450      0.014736     0.015733      0.017848   
            531      0.012040      0.012231     0.014507      0.014973   
            532      0.006251      0.008482     0.007321      0.010120   
            533      0.018088      0.018581     0.021717      0.022744   
            548      0.013194      0.016949     0.014626      0.020291   

                  Autocorr_PE  Autocorr_LNE  Razon_LNE_PE  
EDO DTO MUN SECC                                           
1   1   2   338      0.689191      0.581325      0.991580  
            339      0.717164      0.158740      0.990970  
            340      0.827905      0.152959      0.989805  
            341      0.635793      0.435180      0.987758  
            342      0.634410      0.367652      0.984600  
...                       ...           ...           ...  
32  4   57  530      0.955188      0.952950      0.988422  
            531      0.974564      0.961922      0.989747  
            532      0.934510      0.681005      0.986724  
            533      0.985049      0.928739      0.986229  
            548      0.598121      0.835766      0.990182  

[68428 rows x 7 columns]

In [26]:
#se recupera el índice (desagrupa)
datos2019_secc= datos2019_secc.reset_index()
datos2019_secc

,EDO,DTO,MUN,SECC,Coef_Var_PE,Coef_Var_LNE,Var_Prop_PE,Var_Prop_LNE,Autocorr_PE,Autocorr_LNE,Razon_LNE_PE
0,1,1,2,338,0.009274,0.014711,0.011123,0.017569,0.689191,0.581325,0.991580
1,1,1,2,339,0.006067,0.010667,0.007416,0.010046,0.717164,0.158740,0.990970
2,1,1,2,340,0.007424,0.013348,0.008319,0.013217,0.827905,0.152959,0.989805
3,1,1,2,341,0.006535,0.012118,0.007678,0.014344,0.635793,0.435180,0.987758
4,1,1,2,342,0.013673,0.020742,0.016305,0.024695,0.634410,0.367652,0.984600
...,...,...,...,...,...,...,...,...,...,...,...
68423,32,4,57,530,0.013450,0.014736,0.015733,0.017848,0.955188,0.952950,0.988422
68424,32,4,57,531,0.012040,0.012231,0.014507,0.014973,0.974564,0.961922,0.989747
68425,32,4,57,532,0.006251,0.008482,0.007321,0.010120,0.934510,0.681005,0.986724
68426,32,4,57,533,0.018088,0.018581,0.021717,0.022744,0.985049,0.928739,0.986229


In [27]:
#se agregan los datos de PE, LNE (diciembre2019) y la tasa de crecimiento anual 2019 de la LNE
datos2019_secc_vf = pd.merge(datos2019_secc, diciembre_2019, on=['EDO','DTO','MUN','SECC'], how='inner')
datos2019_secc_vf

,EDO,DTO,MUN,SECC,Coef_Var_PE,Coef_Var_LNE,Var_Prop_PE,Var_Prop_LNE,Autocorr_PE,Autocorr_LNE,Razon_LNE_PE,PE,LNE,TC_LNE_2019
0,1,1,2,338,0.009274,0.014711,0.011123,0.017569,0.689191,0.581325,0.991580,1786.0,1770.0,-0.011173
1,1,1,2,339,0.006067,0.010667,0.007416,0.010046,0.717164,0.158740,0.990970,1724.0,1707.0,0.008269
2,1,1,2,340,0.007424,0.013348,0.008319,0.013217,0.827905,0.152959,0.989805,1771.0,1751.0,0.019802
3,1,1,2,341,0.006535,0.012118,0.007678,0.014344,0.635793,0.435180,0.987758,2074.0,2041.0,0.002456
4,1,1,2,342,0.013673,0.020742,0.016305,0.024695,0.634410,0.367652,0.984600,1559.0,1520.0,0.004627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68423,32,4,57,530,0.013450,0.014736,0.015733,0.017848,0.955188,0.952950,0.988422,1671.0,1649.0,0.039723
68424,32,4,57,531,0.012040,0.012231,0.014507,0.014973,0.974564,0.961922,0.989747,1601.0,1581.0,0.036721
68425,32,4,57,532,0.006251,0.008482,0.007321,0.010120,0.934510,0.681005,0.986724,1081.0,1065.0,0.022073
68426,32,4,57,533,0.018088,0.018581,0.021717,0.022744,0.985049,0.928739,0.986229,1183.0,1164.0,0.052441


In [28]:
#se unen los datos de area y tipo de sección
data_area_tipo = pd.merge(data_tipo_secc, data_area, on=['EDO','DTO','MUN','SECC'], how='inner')
data_area_tipo

,EDO,DTO,MUN,SECC,TIPO_SECCION,Area_Km2
0,1,1,2,338,M,0.506492
1,1,1,2,339,M,0.692053
2,1,1,2,340,R,0.990536
3,1,1,2,341,M,0.507441
4,1,1,2,342,R,1.437017
...,...,...,...,...,...,...
61035,32,4,57,529,U,0.545010
61036,32,4,57,530,M,0.757383
61037,32,4,57,531,U,0.548031
61038,32,4,57,532,U,0.149622


In [29]:
#se unen los datos de area_tipo sección con los datos de feature engineering
datos2019_secc_vf2 = pd.merge(datos2019_secc_vf, data_area_tipo, on=['EDO','DTO','MUN','SECC'], how='inner')
datos2019_secc_vf2

,EDO,DTO,MUN,SECC,Coef_Var_PE,Coef_Var_LNE,Var_Prop_PE,Var_Prop_LNE,Autocorr_PE,Autocorr_LNE,Razon_LNE_PE,PE,LNE,TC_LNE_2019,TIPO_SECCION,Area_Km2
0,1,1,2,338,0.009274,0.014711,0.011123,0.017569,0.689191,0.581325,0.991580,1786.0,1770.0,-0.011173,M,0.506492
1,1,1,2,339,0.006067,0.010667,0.007416,0.010046,0.717164,0.158740,0.990970,1724.0,1707.0,0.008269,M,0.692053
2,1,1,2,340,0.007424,0.013348,0.008319,0.013217,0.827905,0.152959,0.989805,1771.0,1751.0,0.019802,R,0.990536
3,1,1,2,341,0.006535,0.012118,0.007678,0.014344,0.635793,0.435180,0.987758,2074.0,2041.0,0.002456,M,0.507441
4,1,1,2,342,0.013673,0.020742,0.016305,0.024695,0.634410,0.367652,0.984600,1559.0,1520.0,0.004627,R,1.437017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61002,32,4,57,529,0.016715,0.017088,0.019741,0.020404,0.977450,0.920976,0.987773,1740.0,1703.0,0.049291,U,0.545010
61003,32,4,57,530,0.013450,0.014736,0.015733,0.017848,0.955188,0.952950,0.988422,1671.0,1649.0,0.039723,M,0.757383
61004,32,4,57,531,0.012040,0.012231,0.014507,0.014973,0.974564,0.961922,0.989747,1601.0,1581.0,0.036721,U,0.548031
61005,32,4,57,532,0.006251,0.008482,0.007321,0.010120,0.934510,0.681005,0.986724,1081.0,1065.0,0.022073,U,0.149622


In [30]:
#calculamos la densidad por seccion (al corte de diciembre 2019)
datos2019_secc_vf2['Densidad_LNE'] = datos2019_secc_vf2['LNE']/datos2019_secc_vf2['Area_Km2']
datos2019_secc_vf2

,EDO,DTO,MUN,SECC,Coef_Var_PE,Coef_Var_LNE,Var_Prop_PE,Var_Prop_LNE,Autocorr_PE,Autocorr_LNE,Razon_LNE_PE,PE,LNE,TC_LNE_2019,TIPO_SECCION,Area_Km2,Densidad_LNE
0,1,1,2,338,0.009274,0.014711,0.011123,0.017569,0.689191,0.581325,0.991580,1786.0,1770.0,-0.011173,M,0.506492,3494.628518
1,1,1,2,339,0.006067,0.010667,0.007416,0.010046,0.717164,0.158740,0.990970,1724.0,1707.0,0.008269,M,0.692053,2466.573479
2,1,1,2,340,0.007424,0.013348,0.008319,0.013217,0.827905,0.152959,0.989805,1771.0,1751.0,0.019802,R,0.990536,1767.730639
3,1,1,2,341,0.006535,0.012118,0.007678,0.014344,0.635793,0.435180,0.987758,2074.0,2041.0,0.002456,M,0.507441,4022.144814
4,1,1,2,342,0.013673,0.020742,0.016305,0.024695,0.634410,0.367652,0.984600,1559.0,1520.0,0.004627,R,1.437017,1057.746932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61002,32,4,57,529,0.016715,0.017088,0.019741,0.020404,0.977450,0.920976,0.987773,1740.0,1703.0,0.049291,U,0.545010,3124.714237
61003,32,4,57,530,0.013450,0.014736,0.015733,0.017848,0.955188,0.952950,0.988422,1671.0,1649.0,0.039723,M,0.757383,2177.233490
61004,32,4,57,531,0.012040,0.012231,0.014507,0.014973,0.974564,0.961922,0.989747,1601.0,1581.0,0.036721,U,0.548031,2884.873238
61005,32,4,57,532,0.006251,0.008482,0.007321,0.010120,0.934510,0.681005,0.986724,1081.0,1065.0,0.022073,U,0.149622,7117.946288


## Metadatos y datos procesados

In [31]:
#se combinan las nomenclaturas de estados y municipios
nomenclatura=pd.merge(data_nom_edo,data_nom_mun, on=['EDO'], how='inner')
nomenclatura

,EDO,NOMBRE_ESTADO,MUN,NOMBRE_MUNICIPIO
0,1,AGUASCALIENTES,10,SAN FRANCISCO DE LOS ROMO
1,1,AGUASCALIENTES,8,SAN JOSE DE GRACIA
2,1,AGUASCALIENTES,9,TEPEZALA
3,1,AGUASCALIENTES,11,EL LLANO
4,1,AGUASCALIENTES,2,ASIENTOS
...,...,...,...,...
2453,32,ZACATECAS,45,TEPECHITLAN
2454,32,ZACATECAS,14,GENERAL FRANCISCO R. MURGUIA
2455,32,ZACATECAS,44,TABASCO
2456,32,ZACATECAS,9,CHALCHIHUITES


In [32]:
#se concatenan los nombre de los estados y municipios con los resultados:
datos_procesados=pd.merge(nomenclatura, datos2019_secc_vf2, on=['EDO','MUN'], how='inner')
datos_procesados

,EDO,NOMBRE_ESTADO,MUN,NOMBRE_MUNICIPIO,DTO,SECC,Coef_Var_PE,Coef_Var_LNE,Var_Prop_PE,Var_Prop_LNE,Autocorr_PE,Autocorr_LNE,Razon_LNE_PE,PE,LNE,TC_LNE_2019,TIPO_SECCION,Area_Km2,Densidad_LNE
0,1,AGUASCALIENTES,10,SAN FRANCISCO DE LOS ROMO,1,404,0.010392,0.013701,0.011281,0.015822,0.547527,0.494467,0.992982,804.0,796.0,-0.008717,R,0.255615,3114.052233
1,1,AGUASCALIENTES,10,SAN FRANCISCO DE LOS ROMO,1,470,0.009898,0.015790,0.011420,0.018778,0.606021,0.459351,0.986038,1340.0,1311.0,-0.005311,M,0.497880,2633.165586
2,1,AGUASCALIENTES,10,SAN FRANCISCO DE LOS ROMO,1,471,0.006650,0.011982,0.008169,0.014270,0.707951,0.512860,0.989025,1608.0,1582.0,-0.006905,U,0.438174,3610.437533
3,1,AGUASCALIENTES,10,SAN FRANCISCO DE LOS ROMO,1,472,0.010460,0.014940,0.012687,0.018002,0.795567,0.617178,0.990770,1414.0,1394.0,-0.014841,U,0.404232,3448.510713
4,1,AGUASCALIENTES,10,SAN FRANCISCO DE LOS ROMO,1,473,0.011303,0.011558,0.013614,0.013143,0.893573,0.439638,0.990124,8526.0,8412.0,0.027859,M,2.190191,3840.761507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61002,32,ZACATECAS,11,TRINIDAD GARCIA DE LA CADENA,2,350,0.022847,0.026219,0.025671,0.030310,0.955858,0.917811,0.973187,627.0,604.0,0.070922,M,0.672997,897.477471
61003,32,ZACATECAS,11,TRINIDAD GARCIA DE LA CADENA,2,351,0.033336,0.038347,0.036452,0.043636,0.949086,0.946917,0.971766,428.0,418.0,0.120643,U,0.700173,596.995438
61004,32,ZACATECAS,11,TRINIDAD GARCIA DE LA CADENA,2,352,0.022106,0.027715,0.025957,0.032934,0.949893,0.919404,0.973718,407.0,395.0,0.076294,U,0.216138,1827.533786
61005,32,ZACATECAS,11,TRINIDAD GARCIA DE LA CADENA,2,353,0.027591,0.035514,0.031829,0.041410,0.937516,0.939714,0.972815,382.0,374.0,0.119760,M,0.432270,865.200585


In [33]:
#se guarda el último dataframe (datos procesados) en un csv
#datos_procesados.to_csv(r'Datos-procesados_secc2019.csv', index = None)

<br>

## Referencias
- https://www.analyticsvidhya.com/blog/2018/08/dimensionality-reduction-techniques-python/
- https://plot.ly/python/v3/ipython-notebooks/principal-component-analysis/#pca-vs-lda
- https://www.machinelearningplus.com/plots/top-50-matplotlib-visualizations-the-master-plots-python
- https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b
- https://alexanderfabisch.github.io/t-sne-in-scikit-learn.html
- https://medium.com/@indreshbhattacharyya/feature-selection-categorical-feature-selection-boruta-light-gbm-chi-square-bf47e94e2558
- https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
- https://stackoverflow.com/questions/52472757/creating-a-boxplot-facetgrid-in-seaborn-for-python
- https://www.kdnuggets.com/2019/07/10-simple-hacks-speed-data-analysis-python.html
- https://github.com/adam-p/markdown-here/wiki/Markdown-Here-Cheatsheet